In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import pickle
import folium
from datetime import datetime
from scipy import stats                                                                 
from scipy.stats import f_oneway                                                               #ANOVA
from statsmodels.formula.api import ols                                                        #Regression Summary
from statsmodels.stats.multicomp import pairwise_tukeyhsd        
from statsmodels.stats import weightstats as stests                                            #Z-test code
from statsmodels.stats.proportion import proportions_ztest                                     #Proportions z-test
from sklearn.linear_model import LinearRegression                                              #Simple & MultiLinear Regression
from sklearn.feature_selection import RFE, RFECV, SelectKBest, f_regression, mutual_info_regression     
from sklearn.preprocessing import PolynomialFeatures, StandardScaler                           #Multilinear Regression
from sklearn import metrics                                                                    #MAE, MSE
from sklearn.model_selection import train_test_split
from folium.plugins import HeatMap
%matplotlib inline 
pd.set_option('display.max_columns', 300)

In [2]:
%pwd

'/Users/Chaz/Documents/Flatiron/Repository/flatiron-p2-housing'

In [6]:
df = pd.read_csv("kc_house_data_test_features.csv")
print(df.shape)
df.head()

(4322, 21)


,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [46]:
df.isna().mean()

Unnamed: 0             0.0
id                     0.0
date                   0.0
bedrooms               0.0
bathrooms              0.0
                      ... 
grade_8_sqft_living    0.0
grade_9_sqft_living    0.0
floors_grade_7         0.0
floors_grade_8         0.0
floors_grade_9         0.0
Length: 136, dtype: float64

## Cleaning the Holdout Data

In [7]:
#Turn id into a string to prevent any numerical issues.
df['id'] = df['id'].astype('str')

#Turn zipcode into a string to prevent any numerical issues.
df['zipcode'] = df['zipcode'].astype('str')

#Convert the date column to something workable. We'll first use UTC then strip the year out afterwards.
df['date'] = pd.to_datetime(df['date'], utc = True)

#Taking care of the 33 bedroom property by imputing the mean number of bedrooms.
df['bedrooms'] = np.where(df['bedrooms'] > 30, df['bedrooms'].mean(), df['bedrooms'])

#Taking care of any properties with less than .5 baths. Using .75 as that code for baths has a shower.
df['bathrooms'] = np.where(df['bathrooms'] < .75, .75, df['bathrooms'])

## Feature Engineering

In [9]:
#Transforming the data from the date to get and extract the year in order to calcuate the age of the house.
df['year_sold'] = pd.to_datetime(df['date']).dt.date.astype('datetime64[ns]')

#Convert bedrooms to into to account for the mean transformation we did previously for the 33 bedroom value.
df['bedrooms'] = df['bedrooms'].astype(int)

#Create a column for whether or not a property has a basement
df['has_basement'] = np.where(df['sqft_basement'] > 0, 1, 0)

#Feature Engineering to extrapolate the year sold from the convoluted date format.
df['year_sold'] = pd.DatetimeIndex(df['year_sold']).year # extract only year

#Feature Engineering the age of the home
df['age_built'] = (df['year_sold'] - df['yr_built']).astype(int)
df['age_built'] = (df['age_built'] + 6) #bring up to 2021 values

#Feature Engineering the time since the last renovation on home. No renovation will treat build year as the value.
reno_cond = [df['yr_renovated'] == 0, df['yr_renovated'] != 0]
reno_choices = [df['year_sold'] - df['yr_built'], df['year_sold'] - df['yr_renovated']]
df['last_reno'] = np.select(reno_cond, reno_choices)
df['last_reno'] = (df['last_reno'] + 6) #bring up to 2021 values

#Get month property sold
df['month_sold'] = pd.DatetimeIndex(df['date']).month

#Feature Engineering to convert months to seasons (1=Winter, 2=Spring, 3=Summer, 4=Fall)
season_cond = [df['month_sold'] <= 3, (df['month_sold'] >= 4) & (df['month_sold'] <=6), 
               (df['month_sold'] >= 7) & (df['month_sold'] <=9), (df['month_sold'] >= 10) & (df['month_sold'] <=12)]
season_choices = [1, 2, 3, 4] 
df['season_sold'] = np.select(season_cond, season_choices)

#Simple Boolean answer whether the home was renovated as (True/False)
choices_reno_TF = [0, 1] #0 = True
df['was_reno'] = np.select(reno_cond, choices_reno_TF)

#Feature Engineering columns that show the ratio of beds and bath to home
df['bed_sqft_ratio'] = df['bedrooms'] / df['sqft_living']
df['bath_sqft_ratio'] = df['bathrooms'] / df['sqft_living']
df['floors_sqft_ratio'] = df['floors'] / df['sqft_living']

#Feature Engineering for grade values attributed to letter grade 
df['grade_lvl'] = df.grade.apply(lambda x: 'S+' if x == 13 else (
                                           'S' if x == 12 else (
                                           'S-' if x == 11 else (
                                           'A+' if x == 10 else (
                                           'A' if x ==  9 else (
                                           'A-' if x == 8 else (
                                           'B+' if x == 7 else (
                                           'B' if x == 6 else (
                                           'B-' if x == 5 else (
                                           'C+' if x == 4 else (
                                           'C' if x == 3 else (
                                           'C-' if x == 2 else (
                                           'D')))))))))))))

#Age Built by Bins
bins = [-2, 1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 100000]
labels = ['<1', '1-10', '11-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '>100'] # e.g., (1-10]
df['age_built_binned'] = pd.cut(df['age_built'], bins = bins, labels = labels)

## Creating Dummy Variables

In [10]:
#Create dummy variables for grade, waterfront, and zipcode
df = pd.get_dummies(df, columns = ['grade', 'waterfront', 'zipcode', 'season_sold', 'view', 'condition'], drop_first = True)

## Non-linear Transformations

In [11]:
df['floors^2'] = df['floors'] ** 2
df['sqft_above^2'] = df['sqft_above'] ** 2
df['sqft_basement^2'] = df['sqft_basement'] ** 2
df['bath_sqft_ratio^2'] = df['bath_sqft_ratio'] ** 2
df['bed_sqft_ratio^2'] = df['bath_sqft_ratio'] ** 2
df['floors_sqft_ratio^2'] = df['floors_sqft_ratio'] ** 2
df['sqft_living^2'] = df['sqft_living'] ** 2
df['sqft_living15^2'] = df['sqft_living'] ** 2
df['sqft_lot^2'] = df['sqft_lot'] ** 2
df['sqft_lot15^2'] = df['sqft_lot15'] ** 2

## Interaction Features

In [12]:
#Grades 7-9 had the highest sale rates, so we'll create interaction features for those three.
df['reno_basement'] = df['sqft_basement'] * df['was_reno']
df['sqft_reno'] = df['sqft_living'] * df['was_reno']
df['grade_7_sqft_living'] = df['sqft_living'] * df['grade_7']
df['grade_8_sqft_living'] = df['sqft_living'] * df['grade_8']
df['grade_9_sqft_living'] = df['sqft_living'] * df['grade_9']
df['floors_grade_7'] = df['floors'] * df['grade_7']
df['floors_grade_8'] = df['floors'] * df['grade_8']
df['floors_grade_9'] = df['floors'] * df['grade_9']

## Setting Up Best Model to Run on Holdout Data

In [44]:
df.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,year_sold,has_basement,age_built,last_reno,month_sold,was_reno,bed_sqft_ratio,bath_sqft_ratio,floors_sqft_ratio,grade_lvl,age_built_binned,grade_4,grade_5,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,grade_12,grade_13,waterfront_1,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,...,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199,season_sold_2,season_sold_3,season_sold_4,view_1,view_2,view_3,view_4,condition_2,condition_3,condition_4,condition_5,floors^2,sqft_above^2,sqft_basement^2,bath_sqft_ratio^2,bed_sqft_ratio^2,floors_sqft_ratio^2,sqft_living^2,sqft_living15^2,sqft_lot^2,sqft_lot15^2,reno_basement,sqft_reno,grade_7_sqft_living,grade_8_sqft_living,grade_9_sqft_living,floors_grade_7,floors_grade_8,floors_grade_9
0,0,1974300020,2014-08-27 00:00:00+00:00,4,2.50,2270,11500,1.0,1540,730,1967,0,47.7089,-122.241,2020,10918,2014,1,53,53,8,0,0.001762,0.001101,0.000441,A-,50-60,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1.0,2371600,532900,1.212909e-06,1.212909e-06,1.940655e-07,5152900,5152900,132250000,119202724,0,0,0,2270,0,0.0,1.0,0.0
1,1,1974300020,2015-02-18 00:00:00+00:00,4,2.50,2270,11500,1.0,1540,730,1967,0,47.7089,-122.241,2020,10918,2015,1,54,54,2,0,0.001762,0.001101,0.000441,A-,50-60,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1.0,2371600,532900,1.212909e-06,1.212909e-06,1.940655e-07,5152900,5152900,132250000,119202724,0,0,0,2270,0,0.0,1.0,0.0
2,2,3630020380,2014-11-07 00:00:00+00:00,3,2.50,1470,1779,2.0,1160,310,2005,0,47.5472,-121.998,1470,1576,2014,1,15,15,11,0,0.002041,0.001701,0.001361,A-,11-20,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,4.0,1345600,96100,2.892313e-06,2.892313e-06,1.851081e-06,2160900,2160900,3164841,2483776,0,0,0,1470,0,0.0,2.0,0.0
3,3,1771000290,2014-12-03 00:00:00+00:00,3,1.75,1280,16200,1.0,1030,250,1976,0,47.7427,-122.071,1160,10565,2014,1,44,44,12,0,0.002344,0.001367,0.000781,A-,40-50,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1.0,1060900,62500,1.869202e-06,1.869202e-06,6.103516e-07,1638400,1638400,262440000,111619225,0,0,0,1280,0,0.0,1.0,0.0
4,4,5126310470,2015-01-15 00:00:00+00:00,4,2.75,2830,8126,2.0,2830,0,2005,0,47.4863,-122.140,2830,7916,2015,0,16,16,1,0,0.001413,0.000972,0.000707,A-,11-20,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4.0,8008900,0,9.442620e-07,9.442620e-07,4.994444e-07,8008900,8008900,66031876,62663056,0,0,0,2830,0,0.0,2.0,0.0


### Import Pickled Data

In [63]:
infile = open("final_model.pickle",'rb')
model = pickle.load(infile)
infile.close()

In [64]:
infile = open("other_info.pickle",'rb')
other_info = pickle.load(infile)
infile.close()

In [65]:
print(model.intercept_)
print(len(model.coef_))

-239.18612810812266
79


In [83]:
model.coef_

array([-3.13460529e-03,  6.91516387e-02, -3.55952499e-02,  4.75144104e-07,
        1.63936615e-03,  3.58184960e-02,  3.56817475e-02,  2.48930760e-02,
        4.75784902e-03,  1.46692073e+00, -2.07343568e-01,  8.91993971e-05,
        5.23743919e-02,  6.40650075e-02,  2.74814102e-02, -9.46786864e+00,
       -2.53575504e+01, -3.08630977e+01, -4.18122048e+01, -8.32443405e-02,
       -5.45597137e-02,  7.35251997e-02,  2.41493926e-01,  3.64448235e-01,
        6.05073943e-01,  6.15046303e-01,  7.16895368e-01,  8.51801576e-01,
        1.08955206e+00,  4.41245980e-01, -5.37475716e-03,  7.11467469e-02,
        7.81264042e-02,  5.18095808e-02,  5.67956464e-02, -1.50281688e-01,
       -6.48724098e-02, -1.37044358e-01, -2.59795416e-02, -3.04149602e-01,
        1.24827658e-01, -1.02724716e-01, -3.25411296e-03, -2.38288856e-04,
        1.14278493e-01, -3.39527243e-01, -5.81233612e-02,  1.13711318e-01,
        1.26152667e-02, -1.70973637e-01,  4.48930444e-02,  4.89483679e-02,
        4.73598030e-02,  

In [66]:
final_features = other_info[0]

In [67]:
len(final_features)

79

In [74]:
final_answers = np.exp(model.predict(df[final_features]))

In [99]:
final_answers

array([527949.28170966, 544521.62449651, 359799.04454994, ...,
       275052.57644541, 416189.55226854, 276288.91225009])

In [88]:
len(final_answers)

4322

In [107]:
final_data = pd.DataFrame(final_answers)

In [108]:
final_data

,0
0,527949.281710
1,544521.624497
2,359799.044550
3,462625.466838
4,508724.774403
...,...
4317,538166.773073
4318,444496.127430
4319,275052.576445
4320,416189.552269


In [112]:
final_rmse = np.sqrt(metrics.mean_squared_error(df[final_features], final_data))

ValueError: y_true and y_pred have different number of output (79!=1)